In [13]:
import os
import keras
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
from matplotlib import style
from sklearn.externals import joblib

C:\Users\Pritam\Anaconda3\lib\site-packages\sklearn\externals\joblib\__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


# Date Preparation (Time slice)

In [14]:
from datetime import datetime,timedelta
import datetime as dt
from datetime import date

def date_prep():
    N=365*20
    end=datetime.now()
    start=datetime.now()-timedelta(days=N)
    return start, end

# Preparing csvs of stok Historic Data

In [15]:
import pandas_datareader.data as web
def prepare_csv(comp_name):
    start, end = date_prep()
    try:
        df = web.DataReader(comp_name, 'yahoo', start, end)
        df.to_csv('{}.csv'.format(comp_name), index=False)
    except Exception as e:
        print(e)

# Creating the predicting model

In [10]:
'''
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout

def create_model(x_train):
    ####MODEL cretion #####
    model = Sequential()  # define the Keras model
    model.add(
        LSTM(units=240, return_sequences=True, input_shape=(x_train.shape[1], 6)))  # 120 neurons in the hidden layer
    ##return_sequences=True makes LSTM layer to return the full history including outputs at all times
    model.add(Dropout(0.2))
    model.add(LSTM(units=520, return_sequences=True))
    model.add(Dropout(0.2))
    model.add(LSTM(units=520, return_sequences=True))
    model.add(Dropout(0.2))
    model.add(LSTM(units=128, return_sequences=False))
    model.add(Dropout(0.2))
    model.add(Dense(units=1, activation='sigmoid'))
    # adding optimizer and loss function
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

'''

"\nfrom keras.models import Sequential\nfrom keras.layers import LSTM, Dense, Dropout\n\ndef create_model(x_train):\n    ####MODEL cretion #####\n    model = Sequential()  # define the Keras model\n    model.add(\n        LSTM(units=240, return_sequences=True, input_shape=(x_train.shape[1], 6)))  # 120 neurons in the hidden layer\n    ##return_sequences=True makes LSTM layer to return the full history including outputs at all times\n    model.add(Dropout(0.2))\n    model.add(LSTM(units=520, return_sequences=True))\n    model.add(Dropout(0.2))\n    model.add(LSTM(units=520, return_sequences=True))\n    model.add(Dropout(0.2))\n    model.add(LSTM(units=128, return_sequences=False))\n    model.add(Dropout(0.2))\n    model.add(Dense(units=1, activation='sigmoid'))\n    # adding optimizer and loss function\n    model.compile(optimizer='adam', loss='mean_squared_error')\n    return model\n\n"

# Training Model

In [11]:
'''
def train_model(comp_name):
    # reading a csv file
    prepare_csv(comp_name)
    df = pd.read_csv('{}.csv'.format(comp_name))

    df.dropna(how='any', inplace=True)

    training = df

    sc = MinMaxScaler()
    sc.fit(training)
    training_set_scaled = sc.transform(training)
    filename=f'{comp_name}Scale.pkl'
    joblib.dump(sc,filename)
    
    X_train = []
    y1_train = []
    timestamp = 60
    length = len(training)
    
    #####Open   2, High  0, Low   1, Close   3, Adj Close 5####
    ################ adj close
    for i in range(timestamp, length + 1):
        X_train.append(training_set_scaled[i - timestamp:i, ])

    for i in range(timestamp, length + 1):
        y1_train.extend(training_set_scaled[i:i + 1, 5]) #adj close->5
    x_train = []
    y1_train = np.array(y1_train)
    x_train = X_train[:y1_train.shape[0]]
    y1_train = np.reshape(y1_train, (-1, 1))
    x_train = np.array(x_train)
    x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 6))
    model1 = create_model(x_train)
    model1.fit(x_train, y1_train, epochs=5, batch_size=32)  # ,callbacks=[tensorboard])
    # storing the model
    model1.save('StockPredictor{}AdjClose.model'.format(comp_name))
    x_train = []
    y1_train = []

    #################close
    for i in range(timestamp, length + 1):
        X_train.append(training_set_scaled[i - timestamp:i, ])

    for i in range(timestamp, length + 1):
        y1_train.extend(training_set_scaled[i:i + 1, 3]) #close->3
    x_train = []
    y1_train = np.array(y1_train)
    x_train = X_train[:y1_train.shape[0]]
    y1_train = np.reshape(y1_train, (-1, 1))
    x_train = np.array(x_train)
    x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 6))
    model1 = create_model(x_train)
    model1.fit(x_train, y1_train, epochs=5, batch_size=32)  # ,callbacks=[tensorboard])
    # storing the model
    model1.save('StockPredictor{}Close.model'.format(comp_name))
    x_train = []
    y1_train = []

    ############################LOW
    for i in range(timestamp, length + 1):
        X_train.append(training_set_scaled[i - timestamp:i, ])

    for i in range(timestamp, length + 1):
        y1_train.extend(training_set_scaled[i:i + 1, 1]) #low->1
    x_train = []
    y1_train = np.array(y1_train)
    x_train = X_train[:y1_train.shape[0]]
    y1_train = np.reshape(y1_train, (-1, 1))
    x_train = np.array(x_train)
    x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 6))
    model1 = create_model(x_train)
    model1.fit(x_train, y1_train, epochs=5, batch_size=32)  # ,callbacks=[tensorboard])
    # storing the model
    model1.save('StockPredictor{}Low.model'.format(comp_name))
    x_train = []
    y1_train = []
    ##############High
    for i in range(timestamp, length + 1):
        X_train.append(training_set_scaled[i - timestamp:i, ])

    for i in range(timestamp, length + 1):
        y1_train.extend(training_set_scaled[i:i + 1, 0]) #high->0
    x_train = []
    y1_train = np.array(y1_train)
    x_train = X_train[:y1_train.shape[0]]
    y1_train = np.reshape(y1_train, (-1, 1))
    x_train = np.array(x_train)
    x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 6))
    model1 = create_model(x_train)
    model1.fit(x_train, y1_train, epochs=5, batch_size=32)  # ,callbacks=[tensorboard])
    # storing the model
    model1.save('StockPredictor{}High.model'.format(comp_name))
    x_train = []
    y1_train = []
    ################## Open
    for i in range(timestamp, length + 1):
        X_train.append(training_set_scaled[i - timestamp:i, ])

    for i in range(timestamp, length + 1):
        y1_train.extend(training_set_scaled[i:i + 1, 2]) #open->2
    x_train = []
    y1_train = np.array(y1_train)
    x_train = X_train[:y1_train.shape[0]]
    y1_train = np.reshape(y1_train, (-1, 1))
    x_train = np.array(x_train)
    x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 6))
    model1 = create_model(x_train)
    model1.fit(x_train, y1_train, epochs=5, batch_size=32)  # ,callbacks=[tensorboard])
    # storing the model
    model1.save('StockPredictor{}Open.model'.format(comp_name))
    x_train = []
    y1_train = []

    os.remove(comp_name + '.csv')

'''

"\ndef train_model(comp_name):\n    # reading a csv file\n    prepare_csv(comp_name)\n    df = pd.read_csv('{}.csv'.format(comp_name))\n\n    df.dropna(how='any', inplace=True)\n\n    training = df\n\n    sc = MinMaxScaler()\n    sc.fit(training)\n    training_set_scaled = sc.fit_transform(training)\n\n    X_train = []\n    y1_train = []\n    timestamp = 60\n    length = len(training)\n    \n    #####Open   2, High  0, Low   1, Close   3, Adj Close 5####\n    ################ adj close\n    for i in range(timestamp, length + 1):\n        X_train.append(training_set_scaled[i - timestamp:i, ])\n\n    for i in range(timestamp, length + 1):\n        y1_train.extend(training_set_scaled[i:i + 1, 5]) #adj close->5\n    x_train = []\n    y1_train = np.array(y1_train)\n    x_train = X_train[:y1_train.shape[0]]\n    y1_train = np.reshape(y1_train, (-1, 1))\n    x_train = np.array(x_train)\n    x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 6))\n    model1 = create_model(x_tra

In [12]:
'''
train_model('SBIN.NS')
train_model('TCS.NS')
train_model('WIPRO.NS')
train_model('UNITEDBNK.BO')
train_model('RELIANCE.NS')
train_model('^BSESN')
train_model('^NSEI')
'''

"\ntrain_model('SBIN.NS')\ntrain_model('TCS.NS')\ntrain_model('WIPRO.NS')\ntrain_model('UNITEDBNK.BO')\ntrain_model('RELIANCE.NS')\ntrain_model('^BSESN')\ntrain_model('^NSEI')\n"